#### IMPORTING NECESSATRY LIBRARIES AND LINKING THE GOOGLE DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!ls /content/drive/MyDrive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'Colab Notebooks'  'Copy of Mt5.ipynb'	 nlp-final  'Untitled presentation.gslides'


In [ ]:
!pip install bert_score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.4 MB/s eta 0:00:00


In [ ]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# Initialize evaluation metrics
from evaluate import load

bleu = load("bleu")
bertscore = load("bertscore")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from transformers import (
    MBart50Tokenizer,
    MBartForConditionalGeneration,
    MT5Tokenizer,
    MT5ForConditionalGeneration,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    EarlyStoppingCallback
)
from evaluate import load
import torch
import zipfile
import os
import pandas as pd
import re
from sklearn.model_selection import train_test_split

##### DOWNLOADING THE DATASET AND COMBINING THE ENGLISH AND HINSI FILES TO MAKE IT A PARALLEL TRANSLATION DATASET

In [ ]:
# Download and unzip dataset
zip_path = "/content/drive/MyDrive/nlp-final/parallel.zip"
extract_path = "/content/drive/MyDrive/nlp-final/parallel_extracted"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

english_file = os.path.join(extract_path, "parallel-n", "IITB.en-hi.en")
hindi_file = os.path.join(extract_path, "parallel-n", "IITB.en-hi.hi")

with open(english_file, "r", encoding="utf-8") as en_file:
    english_sentences = en_file.readlines()
with open(hindi_file, "r", encoding="utf-8") as hi_file:
    hindi_sentences = hi_file.readlines()

assert len(english_sentences) == len(hindi_sentences)

##### PREPROCESSING

In [ ]:
# Data Preprocessing Functions
def clean_and_add_tokens_english(text):
    sentences = re.split(r'(?<=[\.\?\!])\s+', text.strip())
    cleaned_sentences = ["<s> " + re.sub(r"[^A-Za-z\s\.\?\!]", "", s).lower().strip() + " </s>" for s in sentences if s]
    return " ".join(cleaned_sentences)

def clean_and_add_tokens_hindi(text):
    sentences = re.split(r'(?<=[\।\?\!])\s+', text.strip())
    cleaned_sentences = ["<s> " + re.sub(r"[^\u0900-\u097F\s\?\!]", "", s).strip().replace("।", "") + " </s>" for s in sentences if s]
    return " ".join(cleaned_sentences)

# Apply preprocessing
df = pd.DataFrame({'English': english_sentences, 'Hindi': hindi_sentences})
df['English'] = df['English'].apply(clean_and_add_tokens_english)
df['Hindi'] = df['Hindi'].apply(clean_and_add_tokens_hindi)
df = df[df['English'].apply(lambda x: len(x.split()) > 3)]
df = df[df['Hindi'].apply(lambda x: len(x.split()) > 3)]
train_df, test_df = train_test_split(df, test_size=0.2)

# Print a few preprocessed examples
print("Preprocessed English examples:")
print(train_df['English'].head().tolist())
print("\nPreprocessed Hindi examples:")
print(train_df['Hindi'].head().tolist())


Preprocessed English examples:
['<s> file exists but is not a regular file. </s>', '<s> so i was aroused at . </s> <s> a. </s> <s> m. </s> <s> and taken to the dak bungalow just in time to greet my father who was leaving in the opposite direction. </s>', '<s> star trek </s>', '<s> background color of tasks that are due today in  rrggbb format. </s>', '<s> moly mollinesia sp. </s>']

Preprocessed Hindi examples:
['<s> फ़ाइल मौजूद है लेकिन एक सामान्य फ़ाइल नहीं है </s>', '<s> इसीलिये सबेरे साढे़ पांच बजे मुझे जगाया गया और डाक बंगले ले जाया गया जिससे अपने पिताजी का अभिवादन कर सकूं जो विपरीत दिशा में दौरे के लिये जा रहे थे </s>', '<s> स्टार ट्रेक </s>', '<s> कार्य का पृष्ठभूमि रंग जो आज किया जाना है  प्रारूप में </s>', '<s> मोली मोलीनेसिया स्पीोशिज </s>']


##### EXPORTING THE DATASET FOR FURTHER USE

In [ ]:
# Saving the test and train data as csv
train_path = '/content/drive/My Drive/train_data.csv'
test_path = '/content/drive/My Drive/test_data.csv'

# Export train_df to CSV
train_df.to_csv(train_path, index=False)

# Export test_df to CSV
test_df.to_csv(test_path, index=False)


The TranslationDataset class below serves as a custom PyTorch dataset for machine translation tasks. It tokenizes source and target text pairs using a specified tokenizer, applies padding and truncation to ensure uniform sequence lengths, and formats the data as PyTorch tensors. The target labels are processed to mask padding tokens with -100, ensuring they are ignored during loss computation. This class is compatible with PyTorch's DataLoader, enabling efficient batching and shuffling for training transformer models.

In [ ]:
# Translation Dataset class
class TranslationDataset(torch.utils.data.Dataset):
    def __init__(self, source_texts, target_texts, tokenizer, max_length=128):
        self.source_texts = source_texts
        self.target_texts = target_texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.source_texts)

    def __getitem__(self, idx):
        source = self.source_texts[idx]
        target = self.target_texts[idx]
        source_encodings = self.tokenizer(source, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt")
        target_encodings = self.tokenizer(target, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt")
        labels = target_encodings["input_ids"]
        labels[labels == self.tokenizer.pad_token_id] = -100
        return {"input_ids": source_encodings["input_ids"].flatten(),
                "attention_mask": source_encodings["attention_mask"].flatten(),
                "labels": labels.flatten()}

The evaluate_translations function below evaluates a translation model by generating translations for input texts and comparing them to reference translations using BLEU (n-gram overlap) and BERTScore (semantic similarity). It returns the BLEU score and BERTScore results, providing insights into the model's translation quality.

In [ ]:
# Evaluation function with BLEU and BERTScore
def evaluate_translations(texts, references, model, tokenizer, lang):
    translations = []
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True).to(model.device)
        outputs = model.generate(inputs["input_ids"], max_length=128, num_beams=5, early_stopping=True)
        translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
        translations.append(translation)
    bleu_score = bleu.compute(predictions=translations, references=references)
    bertscore_results = bertscore.compute(predictions=translations, references=references, lang=lang)
    return bleu_score, bertscore_results

##### VAMSHI'S MODEL -  mBART English-to-Hindi Translation

In [ ]:
# Vamshi: mBART English-to-Hindi Translation
mbart_tokenizer_en_hi = MBart50Tokenizer.from_pretrained('facebook/mbart-large-50-many-to-many-mmt', src_lang="en_XX", tgt_lang="hi_IN")
mbart_model_en_hi = MBartForConditionalGeneration.from_pretrained('facebook/mbart-large-50-many-to-many-mmt')
train_dataset_en_hi = TranslationDataset(train_df['English'].tolist(), train_df['Hindi'].tolist(), mbart_tokenizer_en_hi)
eval_dataset_en_hi = TranslationDataset(test_df['English'].tolist(), test_df['Hindi'].tolist(), mbart_tokenizer_en_hi)



training_args_vamshi = TrainingArguments(
    output_dir="./results_en_hi",
    eval_strategy="epoch",
    per_device_train_batch_size=32,
    learning_rate=1e-5,                   # Lower learning rate
    num_train_epochs=3,
    max_steps=10000,
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16=True,
    max_grad_norm=1.0,                    # Gradient clipping
    lr_scheduler_type="linear",           # Learning rate scheduler
    warmup_steps=500,                     # Warmup steps for scheduler
    gradient_accumulation_steps=4,         # Gradient accumulation
    report_to="none"
)


trainer_vamshi = Trainer(
    model=mbart_model_en_hi,
    args=training_args_vamshi,
    train_dataset=train_dataset_en_hi,
    eval_dataset=eval_dataset_en_hi,
    data_collator=DataCollatorWithPadding(tokenizer=mbart_tokenizer_en_hi),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)
trainer_vamshi.train()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


Epoch,Training Loss,Validation Loss
0,1.435000,1.368829
1,1.390100,1.363252


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#sav

TrainOutput(global_step=10000, training_loss=1.496608563232422, metrics={'train_runtime': 13436.1911, 'train_samples_per_second': 95.265, 'train_steps_per_second': 0.744, 'total_flos': 3.46739679304876e+17, 'train_loss': 1.496608563232422, 'epoch': 1.2278224568727363})

##### Akshay's Model: mBART Hindi-to-English Translation

In [ ]:
# Akshay: mBART Hindi-to-English Translation
mbart_tokenizer_hi_en = MBart50Tokenizer.from_pretrained('facebook/mbart-large-50-many-to-many-mmt', src_lang="hi_IN", tgt_lang="en_XX")
mbart_model_hi_en = MBartForConditionalGeneration.from_pretrained('facebook/mbart-large-50-many-to-many-mmt')
train_dataset_hi_en = TranslationDataset(train_df['Hindi'].tolist(), train_df['English'].tolist(), mbart_tokenizer_hi_en)
eval_dataset_hi_en = TranslationDataset(test_df['Hindi'].tolist(), test_df['English'].tolist(), mbart_tokenizer_hi_en)

# Updated Training Arguments for Vanishing Gradient Prevention
training_args_akshay = TrainingArguments(
    output_dir="./results_hi_en",
    eval_strategy="epoch",
    per_device_train_batch_size=32,
    learning_rate=1e-5,                   # Lower learning rate to stabilize gradients
    num_train_epochs=3,
    max_steps=10000,
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16=True,
    max_grad_norm=1.0,                    # Gradient clipping
    lr_scheduler_type="linear",           # Use linear scheduler with warmup
    warmup_steps=500,                     # Number of warmup steps
    gradient_accumulation_steps=4,         # Accumulate gradients over multiple steps
    report_to="none"
)

trainer_akshay = Trainer(
    model=mbart_model_hi_en,
    args=training_args_akshay,
    train_dataset=train_dataset_hi_en,
    eval_dataset=eval_dataset_hi_en,
    data_collator=DataCollatorWithPadding(tokenizer=mbart_tokenizer_hi_en),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

# Train Akshay's model
trainer_akshay.train()


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


Epoch,Training Loss,Validation Loss
0,1.298800,1.245559
1,1.257700,1.241941


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#sav

TrainOutput(global_step=10000, training_loss=1.3589445068359376, metrics={'train_runtime': 13466.1733, 'train_samples_per_second': 95.053, 'train_steps_per_second': 0.743, 'total_flos': 3.46739679304876e+17, 'train_loss': 1.3589445068359376, 'epoch': 1.2278224568727363})

##### NIKITAS MODEL - mT5 English to Hindi

In [ ]:
import re
import pandas as pd

# Data Preprocessing Functions for mT5 (without start and end tokens)
def preprocess_english_mT5(text):
    sentences = re.split(r'(?<=[\.\?\!])\s+', text.strip())
    cleaned_sentences = [re.sub(r"[^A-Za-z\s\.\?\!]", "", s).lower().strip() for s in sentences if s]
    return " ".join(cleaned_sentences)

def preprocess_hindi_mT5(text):
    sentences = re.split(r'(?<=[\।\?\!])\s+', text.strip())
    cleaned_sentences = [re.sub(r"[^\u0900-\u097F\s\?\!]", "", s).strip().replace("।", "") for s in sentences if s]
    return " ".join(cleaned_sentences)


In [ ]:
# Apply preprocessing functions to English and Hindi columns
train_df['English'] = train_df['English'].apply(preprocess_english_mT5)
train_df['Hindi'] = train_df['Hindi'].apply(preprocess_hindi_mT5)

test_df['English'] = test_df['English'].apply(preprocess_english_mT5)
test_df['Hindi'] = test_df['Hindi'].apply(preprocess_hindi_mT5)

# Print a few examples to verify preprocessing
print("Preprocessed Training Data (English):", train_df['English'].head())
print("Preprocessed Training Data (Hindi):", train_df['Hindi'].head())


Preprocessed Training Data (English): 101786            s file exists but is not a regular file. s
1463749    s so i was aroused at . s s a. s s m. s s and ...
585420                                         s star trek s
95238      s background color of tasks that are due today...
1156703                              s moly mollinesia sp. s
Name: English, dtype: object
Preprocessed Training Data (Hindi): 101786         फ़ाइल मौजूद है लेकिन एक सामान्य फ़ाइल नहीं है
1463749    इसीलिये सबेरे साढे़ पांच बजे मुझे जगाया गया और...
585420                                           स्टार ट्रेक
95238      कार्य का पृष्ठभूमि रंग जो आज किया जाना है  प्र...
1156703                             मोली मोलीनेसिया स्पीोशिज
Name: Hindi, dtype: object


In [ ]:
import re
import pandas as pd
from sklearn.model_selection import train_test_split
# Data Preprocessing Functions for mT5 (without manual addition of <s> and </s> tokens)

def clean_english_text(text):
    # Remove all occurrences of <s> and </s> tags
    text = re.sub(r"</?s>", "", text)
    # Split into sentences based on punctuation marks
    sentences = re.split(r'(?<=[.!?])\s+', text.strip())
    # Clean each sentence by removing non-alphabetic characters, except for sentence punctuation
    cleaned_sentences = [re.sub(r"[^A-Za-z\s.!?]", "", s).lower().strip() for s in sentences if s]
    # Join sentences, remove any isolated 's' left from previous tags, and collapse extra spaces
    cleaned_text = " ".join(cleaned_sentences)
    cleaned_text = re.sub(r"\b[s]\b", "", cleaned_text)
    # Remove extra whitespaces and trim any trailing space
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

def clean_hindi_text(text):
    # Remove all occurrences of <s> and </s> tags
    text = re.sub(r"</?s>", "", text)
    # Split into sentences using Hindi-specific punctuation
    sentences = re.split(r'(?<=[।!?])\s+', text.strip())
    # Clean each sentence by retaining only Devanagari script characters and sentence-ending punctuation
    cleaned_sentences = [re.sub(r"[^\u0900-\u097F\s!?]", "", s).replace("।", "").strip() for s in sentences if s]
    # Join sentences, remove extra whitespaces, and trim any trailing space
    cleaned_text = " ".join(cleaned_sentences)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

train_df['English'] = train_df['English'].apply(clean_english_text)
train_df['Hindi'] = train_df['Hindi'].apply(clean_hindi_text)

test_df['English'] = test_df['English'].apply(clean_english_text)
test_df['Hindi'] = test_df['Hindi'].apply(clean_hindi_text)

print("Preprocessed English examples:")
print(train_df['English'].head().tolist())
print("\nPreprocessed Hindi examples:")
print(train_df['Hindi'].head().tolist())


Preprocessed English examples:
['file exists but is not a regular file.', 'so i was aroused at . a. m. and taken to the dak bungalow just in time to greet my father who was leaving in the opposite direction.', 'star trek', 'background color of tasks that are due today in rrggbb format.', 'moly mollinesia sp.']

Preprocessed Hindi examples:
['फ़ाइल मौजूद है लेकिन एक सामान्य फ़ाइल नहीं है', 'इसीलिये सबेरे साढे़ पांच बजे मुझे जगाया गया और डाक बंगले ले जाया गया जिससे अपने पिताजी का अभिवादन कर सकूं जो विपरीत दिशा में दौरे के लिये जा रहे थे', 'स्टार ट्रेक', 'कार्य का पृष्ठभूमि रंग जो आज किया जाना है प्रारूप में', 'मोली मोलीनेसिया स्पीोशिज']


In [ ]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer, Trainer, TrainingArguments, DataCollatorForSeq2Seq

import torch

class TranslationDataset_mt5(torch.utils.data.Dataset):
    def __init__(self, source_texts, target_texts, tokenizer, max_length=128):
        self.source_texts = source_texts
        self.target_texts = target_texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.source_texts)

    def __getitem__(self, idx):
        source = self.source_texts[idx]
        target = self.target_texts[idx]

        # Tokenize source and target with padding and truncation
        # Adjust tokenization in the dataset class to let the tokenizer handle special tokens
        source_encodings = self.tokenizer(
            source, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt", add_special_tokens=True
        )
        target_encodings = self.tokenizer(
            target, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt", add_special_tokens=True
        )

        # Prepare labels by replacing padding tokens with -100
        labels = target_encodings["input_ids"].squeeze()
        labels[labels == self.tokenizer.pad_token_id] = -100

        # Return tokenized inputs, attention masks, and labels
        return {
            "input_ids": source_encodings["input_ids"].squeeze(),
            "attention_mask": source_encodings["attention_mask"].squeeze(),
            "labels": labels
        }

# Load pre-trained mT5-small model and tokenizer
mt5_tokenizer = MT5Tokenizer.from_pretrained('google/mt5-small')
mt5_model = MT5ForConditionalGeneration.from_pretrained('google/mt5-small')

# Create datasets
train_dataset_mt5 = TranslationDataset_mt5(train_df['English'].tolist(), train_df['Hindi'].tolist(), mt5_tokenizer)
eval_dataset_mt5 = TranslationDataset_mt5(test_df['English'].tolist(), test_df['Hindi'].tolist(), mt5_tokenizer)


for name, param in mt5_model.named_parameters():
    if not param.is_contiguous():
        param.data = param.data.contiguous()
    if name.startswith("decoder"):  # Unfreeze decoder layers
        param.requires_grad = True
    elif name.startswith("encoder") and "embed_tokens" not in name:
        param.requires_grad = False


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Make all parameters contiguous before training
for param in mt5_model.parameters():
    if not param.is_contiguous():
        param.data = param.data.contiguous()


In [ ]:
from transformers import MT5Tokenizer, MT5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorWithPadding
import torch

# Define TranslationDataset class
class TranslationDataset(torch.utils.data.Dataset):
    def __init__(self, source_texts, target_texts, tokenizer, max_length=128):
        self.source_texts = source_texts
        self.target_texts = target_texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.source_texts)

    def __getitem__(self, idx):
        source = self.source_texts[idx]
        target = self.target_texts[idx]
        source_encodings = self.tokenizer(source, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt")
        target_encodings = self.tokenizer(target, truncation=True, padding="max_length", max_length=self.max_length, return_tensors="pt")
        labels = target_encodings["input_ids"]
        labels[labels == self.tokenizer.pad_token_id] = -100
        return {
            "input_ids": source_encodings["input_ids"].flatten(),
            "attention_mask": source_encodings["attention_mask"].flatten(),
            "labels": labels.flatten()
        }

# Initialize mT5 tokenizer and model
mt5_tokenizer = MT5Tokenizer.from_pretrained('google/mt5-small')
mt5_model = MT5ForConditionalGeneration.from_pretrained('google/mt5-small')

# Create TranslationDataset instances for train and test data
train_dataset_mt5 = TranslationDataset(train_df['English'].tolist(), train_df['Hindi'].tolist(), mt5_tokenizer)
eval_dataset_mt5 = TranslationDataset(test_df['English'].tolist(), test_df['Hindi'].tolist(), mt5_tokenizer)

# Define TrainingArguments for Nikita's mT5 model
training_args_nikita = TrainingArguments(
    output_dir="./results_mt5_en_hi",
    evaluation_strategy="steps",
    max_steps=1000,
    per_device_train_batch_size=16,
    learning_rate=3e-6,                    # Lower learning rate
    num_train_epochs=3,                    # Use epochs instead of max_steps
    load_best_model_at_end=True,
    fp16=True,                            # Disable mixed precision to avoid instability
    max_grad_norm=1.0,                     # Reduce gradient clipping threshold
    lr_scheduler_type="linear",
    warmup_steps=1000,
    gradient_accumulation_steps=2,
    logging_dir='./logs',
    logging_steps=100,
    report_to="none"
)

# Initialize Trainer
trainer_nikita = Trainer(
    model=mt5_model,
    args=training_args_nikita,
    train_dataset=train_dataset_mt5,
    eval_dataset=eval_dataset_mt5,
    data_collator=DataCollatorWithPadding(tokenizer=mt5_tokenizer),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Train the model
trainer_nikita.train()


##### SAVING THE MODELS TO DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Save Vamshi's English-to-Hindi mBART Model and Tokenizer
trainer_vamshi.save_model("/content/drive/MyDrive/vamshi_mbart_en_hindi")   # Saves the model in Google Drive
mbart_tokenizer_en_hi.save_pretrained("/content/drive/MyDrive/vamshi_mbart_en_hindi")  # Saves the tokenizer

# Save Akshay's Hindi-to-English mBART Model and Tokenizer
trainer_akshay.save_model("/content/drive/MyDrive/akshay_mbart_hi_english")   # Saves the model in Google Drive
mbart_tokenizer_hi_en.save_pretrained("/content/drive/MyDrive/akshay_mbart_hi_english")  # Saves the tokenizer



In [ ]:
import pandas as pd

# Replace 'data.csv' with your actual file path
train_df = pd.read_csv('/content/drive/My Drive/train_data.csv')
test_df = pd.read_csv('/content/drive/My Drive/test_data.csv')



#Save Nikita's English-to-Hindi mT5 Model and Tokenizer
trainer.save_model("/content/drive/MyDrive/nikita_mt5_en_hi")   # Saves the model in Google Drive
mt5_tokenizer.save_pretrained("/content/drive/MyDrive/nikita_mt5_en_hi")  # Saves the tokenizer

##### EVALUATION OF ALL MODELS

In [ ]:
import torch

# Define the translate_text function for mBART
def translate_text(text, model, tokenizer, source_lang="en_XX", target_lang="hi_IN"):
    # Set the source and target language tokens
    tokenizer.src_lang = source_lang
    tokenizer.tgt_lang = target_lang

    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True).to(model.device)

    # Generate translation with forced BOS token for the target language
    with torch.no_grad():
        translated_tokens = model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=128,
            num_beams=5,
            early_stopping=True,
            forced_bos_token_id=tokenizer.lang_code_to_id[target_lang]
        )

    # Decode the translated tokens to get the final translation
    translation = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return translation


In [ ]:
model_path = "/content/drive/MyDrive/vamshi_mbart_en_hi"

from transformers import MBartForConditionalGeneration, MBart50Tokenizer

# Load the fine-tuned model and tokenizer from Google Drive
mbart_tokenizer_en_hi = MBart50Tokenizer.from_pretrained(model_path)
mbart_model_en_hi = MBartForConditionalGeneration.from_pretrained(model_path).to("cuda" if torch.cuda.is_available() else "cpu")


import torch
from transformers import MBartForConditionalGeneration, MBart50Tokenizer
from evaluate import load

# Define the translate_text function for mBART
def translate_text(text, model, tokenizer, source_lang="en_XX", target_lang="hi_IN"):
    # Set the source and target language tokens
    tokenizer.src_lang = source_lang
    tokenizer.tgt_lang = target_lang

    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True).to(model.device)

    # Generate translation with forced BOS token for the target language
    with torch.no_grad():
        translated_tokens = model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=128,
            num_beams=5,
            early_stopping=True,
            forced_bos_token_id=tokenizer.lang_code_to_id[target_lang]
        )

    # Decode the translated tokens to get the final translation
    translation = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return translation

# Sample test cases for Vamshi's English-to-Hindi mBART Model
sample_texts = [
    "How are you?",
    "This is a test of the translation model.",
    "The weather today is quite pleasant.",
    "Could you please help me?",
    "Machine learning is transforming many industries."
]
reference_hi_translations = [
    "आप कैसे हैं?",
    "यह अनुवाद मॉडल का परीक्षण है।",
    "आज का मौसम काफी सुहावना है।",
    "क्या आप कृपया मेरी मदद कर सकते हैं?",
    "मशीन लर्निंग कई उद्योगों को बदल रही है।"
]



# Generate translations using the translate_text function
translations = [translate_text(text, mbart_model_en_hi, mbart_tokenizer_en_hi, source_lang="en_XX", target_lang="hi_IN") for text in sample_texts]

# Initialize BLEU and BERTScore metrics
bleu = load("bleu")
bertscore = load("bertscore")

# Compute BLEU score
bleu_score = bleu.compute(predictions=translations, references=reference_hi_translations)
print(f"BLEU Score: {bleu_score['bleu']}")


# Compute BERTScore
bertscore_results = bertscore.compute(predictions=translations, references=reference_hi_translations, lang="hi")
print(f"BERTScore: Precision: {bertscore_results['precision']}, Recall: {bertscore_results['recall']}, F1: {bertscore_results['f1']}")

# Print Vamshi's results
print("\nVamshi's English-to-Hindi Model Evaluation:")
for i, (translation, reference) in enumerate(zip(translations, reference_hi_translations)):
    print(f"Input Sentence {i+1}: {sample_texts[i]}")
    print(f"Translation: {translation}")
    print(f"Reference: {reference}")


BLEU Score: 0.5527545614717818


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

BERTScore: Precision: [1.0, 0.9753129482269287, 0.8831307291984558, 0.9821296334266663, 0.9349017143249512], Recall: [1.0, 0.9645156860351562, 0.836595892906189, 0.9093760251998901, 0.8880341053009033], F1: [1.0, 0.9698842763900757, 0.8592337369918823, 0.9443535804748535, 0.9108654856681824]

Vamshi's English-to-Hindi Model Evaluation:
Input Sentence 1: How are you?
Translation: आप कैसे हैं?
Reference: आप कैसे हैं?
Input Sentence 2: This is a test of the translation model.
Translation: यह अनुवाद मॉडल का परीक्षण है
Reference: यह अनुवाद मॉडल का परीक्षण है।
Input Sentence 3: The weather today is quite pleasant.
Translation: आज मौसम काफी सुखद है
Reference: आज का मौसम काफी सुहावना है।
Input Sentence 4: Could you please help me?
Translation: क्या आप मेरी मदद कर सकते हैं?
Reference: क्या आप कृपया मेरी मदद कर सकते हैं?
Input Sentence 5: Machine learning is transforming many industries.
Translation: मशीन शिक्षण कई उद्योगों को बदल रहा है
Reference: मशीन लर्निंग कई उद्योगों को बदल रही है।


In [ ]:
import torch
from transformers import MBartForConditionalGeneration, MBart50Tokenizer
from evaluate import load

# Mount Google Drive to access the saved model
from google.colab import drive
drive.mount('/content/drive')

# Set the path to the fine-tuned Hindi-to-English model saved in Google Drive
model_path = "/content/drive/MyDrive/akshay_mbart_hi_en"  # Change to your specific path if needed

# Define the translate_text function for Hindi-to-English translation
def translate_text(text, model, tokenizer, source_lang="hi_IN", target_lang="en_XX"):
    # Set the source and target language tokens
    tokenizer.src_lang = source_lang
    tokenizer.tgt_lang = target_lang

    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True).to(model.device)

    # Generate translation with forced BOS token for the target language
    with torch.no_grad():
        translated_tokens = model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=128,
            num_beams=5,
            early_stopping=True,
            forced_bos_token_id=tokenizer.lang_code_to_id[target_lang]
        )

    # Decode the translated tokens to get the final translation
    translation = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return translation

# Sample test cases for Akshay's Hindi-to-English mBART Model
sample_hi_texts = [
    "आपका नाम क्या है?",
    "आज का मौसम कैसा है?",
    "यह तकनीक बहुत प्रभावी है।",
    "मुझे आपकी सहायता चाहिए।",
    "कृपया मुझे रास्ता दिखाइए।"
]
reference_en_translations = [
    "What is your name?",
    "How is the weather today?",
    "This technology is very effective.",
    "I need your assistance.",
    "Please show me the way."
]

# Load the fine-tuned mBART model and tokenizer from Google Drive
mbart_tokenizer_hi_en = MBart50Tokenizer.from_pretrained(model_path)
mbart_model_hi_en = MBartForConditionalGeneration.from_pretrained(model_path).to("cuda" if torch.cuda.is_available() else "cpu")

# Generate translations using the translate_text function
translations = [translate_text(text, mbart_model_hi_en, mbart_tokenizer_hi_en, source_lang="hi_IN", target_lang="en_XX") for text in sample_hi_texts]

# Initialize BLEU and BERTScore metrics
bleu = load("bleu")
bertscore = load("bertscore")

# Compute BLEU score
bleu_score = bleu.compute(predictions=translations, references=reference_en_translations)
print(f"BLEU Score: {bleu_score['bleu']}")

# Compute BERTScore
bertscore_results = bertscore.compute(predictions=translations, references=reference_en_translations, lang="en")
print(f"BERTScore: Precision: {bertscore_results['precision']}, Recall: {bertscore_results['recall']}, F1: {bertscore_results['f1']}")

# Print Akshay's results
print("\nAkshay's Hindi-to-English Model Evaluation:")
for i, (translation, reference) in enumerate(zip(translations, reference_en_translations)):
    print(f"Input Sentence {i+1}: {sample_hi_texts[i]}")
    print(f"Translation: {translation}")
    print(f"Reference: {reference}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
BLEU Score: 0.47479004900952215


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore: Precision: [0.9988645315170288, 0.9597671031951904, 0.9811028242111206, 0.982812225818634, 0.999146580696106], Recall: [0.9988645315170288, 0.9844760894775391, 0.9811028242111206, 0.982812225818634, 0.999146580696106], F1: [0.9988645315170288, 0.9719645977020264, 0.9811028242111206, 0.982812225818634, 0.999146580696106]

Akshay's Hindi-to-English Model Evaluation:
Input Sentence 1: आपका नाम क्या है?
Translation: what is your name?
Reference: What is your name?
Input Sentence 2: आज का मौसम कैसा है?
Translation: what is the weather like today?
Reference: How is the weather today?
Input Sentence 3: यह तकनीक बहुत प्रभावी है।
Translation: this technique is very effective.
Reference: This technology is very effective.
Input Sentence 4: मुझे आपकी सहायता चाहिए।
Translation: i need your help.
Reference: I need your assistance.
Input Sentence 5: कृपया मुझे रास्ता दिखाइए।
Translation: please show me the way.
Reference: Please show me the way.


In [ ]:

import torch
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
from evaluate import load

# Load the fine-tuned model and tokenizer from Google Drive
model_path = "/content/drive/MyDrive/nikita_mt5_en_hi"
mt5_tokenizer_en_hi = MT5Tokenizer.from_pretrained(model_path)
mt5_model_en_hi = MT5ForConditionalGeneration.from_pretrained(model_path).to("cuda" if torch.cuda.is_available() else "cpu")

# Define the translate_text function for mT5 with improved generation settings
def translate_text(text, model, tokenizer, source_lang="en_XX", target_lang="hi_IN"):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True).to(model.device)

    # Generate translation using beam search with adjusted parameters
    with torch.no_grad():
        translated_tokens = model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=128,
            num_beams=5,
            length_penalty=1.0,
            early_stopping=True,
            no_repeat_ngram_size=2,
            top_k=50,
            top_p=0.95
        )

    # Decode the translated tokens to get the final translation, skipping special tokens
    translation = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    translation = translation.replace("<s>", "").strip()
    return translation

# Sample test cases for Nikita's English-to-Hindi mt5 Model
sample_en_texts = [
    
    "I am learning Hindi",
    
    "This is a test of the translation model.",
    
    "Could you please help me?"
    
]
reference_hi_translations = [
    
    "मैं हिंदी सीख रहा हूँ",
    
    "यह अनुवाद मॉडल का परीक्षण है।",
    
    "क्या आप कृपया मेरी मदद कर सकते हैं?"
    
]

# Generate translations
translations = [translate_text(text, mt5_model_en_hi, mt5_tokenizer_en_hi) for text in sample_en_texts]

# Initialize BLEU and BERTScore metrics
bleu = load("bleu")
bertscore = load("bertscore")

# Format the references as a list of lists for BLEU
formatted_references = [[ref] for ref in reference_hi_translations]

# Compute BLEU score
bleu_score = bleu.compute(predictions=translations, references=formatted_references, smooth=True)
print(f"BLEU Score: {bleu_score['bleu']}")

# Compute BERTScore
bertscore_results = bertscore.compute(predictions=translations, references=reference_hi_translations, lang="hi")

# Print BERTScore
print("\nBERTScore Results:")
print(f"Precision: {bertscore_results['precision']}")
print(f"Recall: {bertscore_results['recall']}")
print(f"F1 Score: {bertscore_results['f1']}")

# Print each sentence's translation and BERTScore F1 score
print("\nNikita's English-to-Hindi Model Evaluation:")
for i, (translation, reference) in enumerate(zip(translations, reference_hi_translations)):
    print(f"Input Sentence {i+1}: {sample_en_texts[i]}")
    print(f"Translation: {translation}")
    print(f"Reference: {reference}")
    print(f"BERTScore (F1): {bertscore_results['f1'][i]:.4f}")
    print(f"Tokenized Translation: {mt5_tokenizer_en_hi.encode(translation)}")
    print(f"Tokenized Reference: {mt5_tokenizer_en_hi.encode(reference)}")
    print()



BERTScore Results:
Precision: [0.8253610134124756, 0.8094627261161804, 0.8047354817390442]
Recall: [0.8682999610900879, 0.7365309000015259, 0.80901038646698]
F1 Score: [0.8462861776351929, 0.771276593208313, 0.806867241859436]

Nikita's English-to-Hindi Model Evaluation:
Input Sentence 1: I am learning Hindi
Translation: मैं हिन्दी में पढ़ना चाहता हूं
Reference: मैं हिंदी सीख रहा हूँ
BERTScore (F1): 0.8463
Tokenized Translation: [9422, 2917, 259, 73820, 844, 38844, 1437, 21544, 1114, 2768, 27886, 1]
Tokenized Reference: [9422, 2917, 50315, 1304, 12010, 3547, 259, 11255, 2768, 48108, 1]

Input Sentence 2: This is a test of the translation model.
Translation: यह एक परीक्षण विधि है
Reference: यह अनुवाद मॉडल का परीक्षण है।
BERTScore (F1): 0.7713
Tokenized Translation: [259, 4620, 1890, 259, 38782, 3225, 259, 59059, 830, 1]
Tokenized Reference: [259, 4620, 9608, 18148, 2701, 9462, 60146, 1163, 259, 38782, 3225, 830, 378, 1]

Input Sentence 3: Could you please help me?
Translation: क्या मैं